In [4]:
import random as rd
import numpy as np
from math import sqrt, floor, log2

Implémentation de RSA sous forme de classe.


In [34]:
class RSA:
    
    # Huge prime number
    _PRIME = 4879228431756730165299385010232837291120885737358336711467496639025522618199839237653918283371408014017658368720147197955013865183708637800199520868317779
    _e = 257
    _k0 = 16
    def __init__(self, k0, n):
        self.k0 = k0
        self.n = n
        pass
    
    @staticmethod
    def getPrime(): return RSA._PRIME
    @staticmethod
    def getE(): return RSA._e
    
    @staticmethod
    def bigPrime(l : int) -> int:
        """bigPrime Génère un nombre premier de 'l' bits
        
        Args:
            l (int): Taille en bit du nombre
        
        Returns:
            int: Nombre premier sur l bits 
        """

        # Génère un nombre aléatoire de l-bits (impair)
        n = rd.getrandbits(l) | 1
        n |= (1 << l - 1)

        # Boucle pour s'assurer que n est premier
        while not RSA.millerRabin(n, 5):
            # Génère un nombre aléatoire de l-bit et fait un OR avec 1
            n = rd.getrandbits(l) | 1   # Obligatoirement impair
            n |= (1 << l - 1)
        return n
    
    @staticmethod
    def SnM(a , b , n : int ): 
        return pow(int(a), int(b), n)
    
    @staticmethod
    def EA(a : int, b : int) -> int:
        """
        EEA Méthode pour déterminer le PGCD de deux nombre entier
        
        Args:
            a (int): Premier nombre entier
            b (int): Second nombre entier
        
        Returns:
            int : Plus grand commun diviseur des nombre a et b
        """

        # Cas de base
        if (b == 0): return a
        if (a == b): return b
        
        # Appel récursif selon le cas 
        if (a > b): return RSA.EA(b, a%b)
        else : return RSA.EA(a, b%a)
    
    @staticmethod
    def EEA(a : int, b : int) -> int:
        """EEA Méthode pour déterminer le PGCD de deux nombres entiers
        
        Args:
            a (int): Premier nombre entier
            b (int): Second nombre entier
        
        Returns:
            int[]: Coefficients et plus grand commun diviseur des nombres a et b
        """
        # S'assure que le nombre 'a' est plus grand
        if (a < b): RSA.EEA(b, a)

        # Initialisation des coefficients := [[x0, x1], [y0, y1]]
        coeff = [[1, 0], [0, 1]]

        # Initialise un tableau pour contenir les coefficients 'x' et 'y' ainsi que le PGCD
        result = [None, None, None]

        # Boucle principale
        while(True) :
            # Détermine q et r
            r = a % b   
            if (r == 0): return result
            q = a // b 

            # Obtient les résultats de la ie itération
            result[0] = coeff[0][0] - q * coeff[0][1]
            result[1] = coeff[1][0] - q * coeff[1][1]
            result[2] = r

            # Modifie les coefficients 
            coeff[0] = [coeff[0][1], result[0]]
            coeff[1] = [coeff[1][1], result[1]]

            # Modifie les valerus de 'a' et 'b' qui seront utilisées pour la prochaine itération
            a = b
            b = r
        return result
    
    @staticmethod
    def hash_function(m : int, l : int):
        """hash_function Retourne un hash de longueur 'l' de 'm'
        
        Args:
            m (int): Message à hasher
            l (int): Longueur du hash 
        """
        if (type(m) == str):
            m = int(m, 2)
        # Convertie le message en nombre
        mhash = bin((m * RSA.getPrime()) % pow(2, l-1))
        return mhash[2:].zfill(l)

    

    @staticmethod
    def millerRabin(n : int, s : int) -> bool:
        """millerRabin Test si un nombre est premier
        
        Args:
            n (int): Nombre à tester
            s (int): Nombre de test
        
        Returns:
            bool: Vrai si n est premier
        """
        # Vérifie si le nombre 'p' est pair; retourne vrai si 'p' vaut 2 et faux sinon
        if (n % 2 == 0) : return n == 2
        if n == 3: return True

        # Initialise r, u et _p
        u, _n = 0, n-1
        # Détermine la valeur de 'u'
        while _n & 1 == 0 :
            u += 1      # Augmente la valeur de 'u'
            _n //= 2    # Division entière de _n par 2

        # Boucle principale
        for i in range(s):
            # Détermine un a aléatoire [2, n-1[
            print(n)
            a = rd.randrange(2, n-1)
            # Détermine 'z' initiale
            z = pow(a, _n, n)

            if (z != 1 and z != n-1): 
                # Boucle pour déterminer si 'n' est composé
                j = 1
                while j < u and z != n-1:
                    # Nouvelle valeur de z
                    z = pow(z, _n, n)
                    if z == 1: return False
                    # Augmente le compteur
                    j += 1
                if z != n-1: return False
        return True
    
    #staticmethod
    def OAEP(self, msg : str) -> str:
        """OAEP Méthode implémentant «l'Optimal Asymmetric Encryption Padding»
        
        Args:
            msg (str): Message à «padder»
            l (int)  : Longueur totale du message «paddé»
        
        Returns:
            str: Message «paddé»
        """
        print("--- OAEP ---")
        k0 = self.k0
        n = self.n
        k1 = n - k0 - len(msg)
        # Genere un padding
        padding = "0" * k1
        # Ajoute le padding
        m = msg + padding
        
        # Génère un nombre aléatoire
        r = bin(rd.randrange(0,pow(2, k0-1)))[2:]
        r = r.zfill(k0)
        
        print("m:", m)
        print("r:", r)
        
        # Hash 'r' à n-k0 bits
        hash_r = RSA.hash_function(r, n-k0)
        
        # X == m XOR hash_r
        X = ''.join(str(int(a) ^ int(b)) for a,b in zip(m, hash_r))
        # Hash 'X'
        hash_X = RSA.hash_function(X, k0)

        # Y == r XOR hash_X
        Y = ''.join(str(int(a) ^ int(b)) for a,b in zip(r, hash_X))
        
        return X + Y
    
    #@staticmethod
    def OAEP_inv(self, XY : str) -> str:
        """OAEP_inv Méthode inverse de «OAEP» et permet de retrouver 'm'
        
        Args:
            XY (str): Message «paddé»
        
        Returns:
            str : Meesage original
        """
        k0 = self.k0
        n = self.n
        # Sépare le message en deux parties: X || Y
        X = XY[:n-k0]
        Y = XY[n-k0:]
        
        # Hash 'X'
        hash_X = RSA.hash_function(X, k0)#[2:].zfill(k0)
        # Retrouve 'r'
        r = ''.join(str(int(a)^int(b)) for a, b in zip(Y, hash_X))
        # Hash 'r'
        hash_r = RSA.hash_function(r, n-k0)#[2:].zfill(l-k0)
        # Retrouve 'msg + padding'
        m = ''.join(str(int(a)^int(b)) for a, b in zip(X, hash_r))
        
        print("m:", m)
        print("r:", r)
        
        return m
    
    #@staticmethod
    def genKeys(self, l : int):
        """genKeys Méthode pour générer les clés 'PK' et 'SK'
        
        Args: 
            l (int): Longueur de 'n'
            
        Returns:
            tuple : Clés PK et SK
        """
        # Obtient la valeur de 'e'
        e = RSA.getE()
        # Initialise p et q
        p, q = e+1, e+1
        
        # Obtient les bonnes valeurs de q et p
        while (p-1)%e == 0: p = RSA.bigPrime(l//2)
        while (q-1)%e == 0: q = RSA.bigPrime(l//2)
        
        # Détermine la valeur de 'n' et 'phi_n'
        n = p * q
        phi_n = (p-1) * (q-1)
        # Détermine la valeur de 'd'
        d = RSA.SnM(e, phi_n-1, n)
        
        # Forme les clés
        PK = [n, e]
        SK = [p, q, d]
        
        return PK, SK

    #@staticmethod
    def exp_CRT(self, C : str, SK : list):
        # Convertie 'C' en valeur numérique
        C_num = int(C, 2)
        
        # Obtient les valeurs p, q et d de la clé SK
        p, q, d = SK
        
        dp, dq = d%(p-1), d%(q-1)
        kp, kq = RSA.SnM(q, p-1, p), RSA.SnM(p, q-1, q)
        xp, xq = C_num%p, C_num%q
        yp, yq = RSA.SnM(xp, dp, p), RSA.SnM(xq, dq, q)
        
        # Obtient le message
        msg = bin(((q*kp)*yp + (p*kq)*yp)%(p*q))[2:]
        return msg
         
    #self, @staticmethod
    def encrypt(msg : str, PK : list):
        """encrypt Méthode pour encrypter un message avec une clé publique
        
        Args:
            msg (str): Message à encrypter
            PK (list): Clé publique
            
        Returns:
            cipher (str): Message encrypté
        """
        # Obtient 'n' et 'e' de la clé publique
        N, e = PK
        # Convertie 'N' en binaire
        n = len(bin(N)[2:])
        # Initialise le cipher
        cipher = ""
        
        for i in range(len(msg)//n):
            block = msg[i*n : (i+1)*n]
            cipher += bin(RSA.SnM(block, e, N))[2:]
        
        reminder = len(msg)%N
        return cipher
    
    #@staticmethod
    def decrypt(self, cipher : str, SK : list):
        """decrypt Méthode pour décrypter un cipher selon une clé privée
        
        Args:
            cipher (str): Meesage à décrypter
            SK (list): Clé privée
        
        Returns:
            msg (str): Message décrypté
        """
        # Détermine le nombre de block à décrypter
        nblock = len(cipher) // (SK[0]*SK[1])
        # Initialise le message
        msg = ""
        
        # Boucle pour décrypter les blocks
        for i in range(nblock):
            block = cipher[i*n, (i+1)*n]
            msg += RSA.exp_CRT(block, SK)
        
        # reminder OAEP
        return msg


Test de la classe

In [35]:
N = 1000000
length = 64
msg = bin(rd.randrange(0, pow(2, 31)))[2:]


Rsa = RSA(16, 64)

XY = Rsa.OAEP(msg)
m = Rsa.OAEP_inv(XY)



        


--- OAEP ---
m: 110010110101000111110010100010100000000000000000
r: 0110001100001110
m: 110010110101000111110010100010100000000000000000
r: 0110001100001110
